<a href="https://colab.research.google.com/github/anne-urai/ddm_mediation/blob/main/pyddm_fits2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# First, set Runtime -> Runtime type -> GPU for fitting

# https://hddm.readthedocs.io/en/latest/lan_tutorial.html
!pip install seaborn
!pip install pyddm
!pip install pathos

     |████████████████████████████████| 81 kB 2.7 MB/s 
     |████████████████████████████████| 65 kB 2.4 MB/s 


In [6]:
import os, itertools
import copy
import numpy as np
import scipy as sp
from scipy import stats
import pandas as pd
import statsmodels.api as sm
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed
from IPython import embed as shell

import ddm
from ddm import Sample
from ddm import plot
from ddm import models
from ddm import Model, Fittable, Fitted, Bound, Overlay, Solution
from ddm.models.loss import LossFunction
from ddm.functions import fit_adjust_model, display_model
from ddm.models import DriftConstant, NoiseConstant, BoundConstant, OverlayChain, OverlayNonDecision, OverlayPoissonMixture, OverlayUniformMixture, InitialCondition, ICPoint, ICPointSourceCenter, LossBIC, LossRobustBIC
# from ddm import set_N_cpus

In [7]:
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
sns.set(style='ticks', font='Arial', font_scale=1, rc={
    'axes.labelsize': 7,
    'axes.titlesize': 7,
    'xtick.labelsize': 6,
    'ytick.labelsize': 6,
    'legend.fontsize': 6,
    'axes.linewidth': 0.25,
    'xtick.major.width': 0.25,
    'ytick.major.width': 0.25,
    'ytick.major.width': 0.25,
    'ytick.major.width': 0.25,
    'ytick.major.pad' : 2.0,
    'ytick.minor.pad' : 2.0,
    'xtick.major.pad' : 2.0,
    'xtick.minor.pad' : 2.0,
    'axes.labelpad' : 4.0,
    'axes.titlepad' : 6.0,
    } )
sns.plotting_context()
sns.set_palette("tab10")

In [13]:
class StartingPoint(InitialCondition):
    name = 'A starting point.'
    required_parameters = ['z']
    required_conditions = ['X', 'M']
    def get_IC(self, x, dx, conditions):
        pdf = np.zeros(len(x))
        pdf[int(len(pdf)*self.z)] = 1
        return pdf

class DriftStimulusCoding(ddm.models.Drift):
    name = 'Drift'
    required_parameters = ['v', 'b', 'm', 'x']
    required_conditions = ['stimulus', 'X', 'M']

    def get_drift(self, x, conditions, **kwargs):
        
        stim = conditions['stimulus']
        X = conditions['X']
        M = conditions['M']

        # return:
        return (stim*self.v) + (self.b) + (self.x * X) + (self.m * M)
    
class BoundCollapsingHyperbolic(Bound):
    name = 'Hyperbolic collapsing bounds'
    required_parameters = ['a']
    required_conditions = []
    
    def get_bound(self, t, conditions, **kwargs):
        urgency = False
        if urgency:
            return self.a-(self.a*(t/(t+self.u)))
        else:
            return self.a

class NonDecisionTime(Overlay):
    name = 'Non-decision time'
    required_parameters = ['t']
    def apply(self, solution):
        # Unpack solution object
        corr = solution.corr
        err = solution.err
        m = solution.model
        cond = solution.conditions
        undec = solution.undec
        
        shifts = int(self.t/m.dt) # truncate
        # Shift the distribution
        newcorr = np.zeros(corr.shape, dtype=corr.dtype)
        newerr = np.zeros(err.shape, dtype=err.dtype)
        if shifts > 0:
            newcorr[shifts:] = corr[:-shifts]
            newerr[shifts:] = err[:-shifts]
        elif shifts < 0:
            newcorr[:shifts] = corr[-shifts:]
            newerr[:shifts] = err[-shifts:]
        else:
            newcorr = corr
            newerr = err
        return Solution(newcorr, newerr, m, cond, undec)

def make_model(sample, model_settings):
    
    T_dur = model_settings['T_dur']

    # limits:
    ranges = {
            'z':(0.05,0.95),               # starting point
            'v':(0,5),                     # drift rate
            'b':(-5,5),                    # drift bias
            'a':(0.1,5),                   # bound
            'u':(0.01,T_dur*10),           # hyperbolic collapse
            't':(0,2),                     # non-decision time
            'm':(-100,100),                # non-decision time
            'x':(-100,100),                # non-decision time
            }

    # put together:
    model = Model(name='stimulus coding model / collapsing bound',
                IC=StartingPoint(**{param:Fittable(minval=ranges[param[0]][0], maxval=ranges[param[0]][1]) for param in StartingPoint.required_parameters}),
                drift=DriftStimulusCoding(**{param:Fittable(minval=ranges[param[0]][0], maxval=ranges[param[0]][1]) for param in DriftStimulusCoding.required_parameters}),
                bound=BoundCollapsingHyperbolic(**{param:Fittable(minval=ranges[param[0]][0], maxval=ranges[param[0]][1]) for param in BoundCollapsingHyperbolic.required_parameters}),
                overlay=OverlayChain(overlays=[NonDecisionTime(**{param:Fittable(minval=ranges[param[0]][0], maxval=ranges[param[0]][1]) for param in NonDecisionTime.required_parameters}),
                                                # OverlayUniformMixture(umixturecoef=0)]),
                                                # OverlayPoissonMixture(pmixturecoef=.05, rate=1)
                                                ]),
                noise=NoiseConstant(noise=1),
                dx=.005, dt=.01, T_dur=T_dur)
    return model

def fit_model(df, model_settings):

    from ddm import set_N_cpus
    set_N_cpus(20)

    # sample:
    sample = Sample.from_pandas_dataframe(df=df, rt_column_name='rt', correct_column_name='response')

    # make model
    model = make_model(sample=sample, model_settings=model_settings)

    # fit:
    # model = fit_adjust_model(sample=sample, model=model, lossfunction=LossBIC, fitparams={'maxiter':5000})
    model = fit_adjust_model(sample=sample, model=model, lossfunction=LossRobustBIC)

    # get params:
    # param_names = [component.required_parameters for component in model.dependencies]
    # param_names = [item for sublist in param_names for item in sublist]
    param_names = model.get_model_parameter_names()
    params = pd.DataFrame(np.atleast_2d([p.real for p in model.get_model_parameters()]), columns=param_names)
    params['bic'] = model.fitresult.value()

    return params

In [14]:
for eff_x in ['v']: #, 'z', 'vz', 'no']:
  for eff_m in ['v']: #, 'z', 'vz', 'no']:

   # grab datafile from GitHub
    data = pd.read_csv('https://github.com/anne-urai/ddm_mediation/raw/main/data/data_df_X%s_M%s.csv'%(eff_x, eff_m))
    data = data.rename({'S': 'stimulus'}, axis=1)
    data = data[['subj_idx', 'stimulus', 'response', 'rt', 'M', 'X']]
    
    # bin:
    data['M_bin'] = pd.cut(data['M'], bins=10, labels=False)
    data['M'] = data['M_bin'].map(data.groupby(['M_bin'])['M'].mean())

    # compute T_dur:
    T_dur = data['rt'].max()+1

    # set options:
    model_settings = [
        {'fit': 'pyddm', 'depends_on': {'a':None, 'u':None, 'v':None, 't':None, 'z':None, 'b':None, 'k':None}, 
                        'start_bias': True, 'drift_bias': True, 'leak': False, 'urgency':False, 'T_dur':T_dur},
    ]


    groupby = ['subj_idx']
    version = 0
    res = Parallel(n_jobs=64, verbose=1, backend='loky')(delayed(fit_model)
                    (df, model_settings[version]) for ids, df in data.groupby(groupby))
    params = pd.concat(res).reset_index(drop=True)

[Parallel(n_jobs=64)]: Using backend LokyBackend with 64 concurrent workers.


NameError: ignored

ImportError: ignored